## Autograd Mechanics


Torch包是一个高维张量表示与计算框架，其包含了数据结构以及定义在Tensor数据结构上的数学运算，对应于python中的Numpy包，但torch提供在GPU上的运算。深度神经网络是一系列Tensor操作构成的有向无环图，其网路权重的更新可通过梯度下降等方法完成。Pytorch提供了自动求导机制。每一个Tensor都有一个**requires_grad**标签，来指示该Tensor是否需要计算梯度。在计算过程中，当至少有一个输入变量需要计算梯度时，其后续输出都要计算梯度，只有当所有的输入都不需要计算梯度是，输出才不需要计算梯度。

In [1]:
import torch
import torchvision
import torch.optim as optim

In [2]:
x = torch.randn(5,5) # requires_grad = False by default
y = torch.randn(5,5) # requires_grad = False by default
z = torch.randn((5,5),requires_grad = True)
a = x + y
a.requires_grad

False

In [3]:
b = a + z
b.requires_grad

True

**requires_grad**的一个用途就是在fine-tune网络的时候，需要固定一部分层的参数而只更新另一部分。此时将不需要更新的部分的**requires_grad**设为False即可，在这个计算过程中，这些部分便不在保存梯度信息。

In [ ]:
model = torchvision.models.resnet18(pretrained = True)
for param in model.parameters():
    param.requires_grad = False
# Replace the last fully-connected layer
# Parameters of newly constructed modules have requires_grad = True by default
model.fc = nn.Linear(512,100)

# Optimize only the classifier
optimizer = optim.SGD(model.fc.parameters(), lr = 1e-2,momentum = 0.9)

Pytorch是一个动态图模型，每次计算都会重新构建一个新的计算图。Autograd是一个反向自动微分系统，其将各种**Function**表示成一个有向无环图，图的叶子节点是数据输入，根节点是输出，中间节点为**Function**操作，可被用来处理Tensor数据。Torch.Tensor是一种数据结构，其中**requires_grad**属性表示了是否需要计算梯度，**data**属性存储数据本身，**grad_fn**属性存储生成该Tensor的Function。通过前向传播可根据输入计算输出，通过从根节点到叶子节点的反向追踪，则可根据链式法则计算每一个节点的梯度，最后可以根据具体的优化方法更新网络权重。

#### In-place Operation

In-place操作有两个特点：
- 其改变一个Tensor的值的时候，不经过复制操作，而是直接在原来的内存上改变它的值
- in-place操作会重写计算图，out-of-place部分会分配新的object,其和原图具有相同的引用，但是in-place操作需要该Function的所有输入Tensor的creater改为此Function。如果被修改的Tensor被其他Tensor所引用，in-place操作将会抛出错误。

因此，一般不建议在神经网络构建过程中使用in-place操作，一方面当前的计算机资源足够用，除非在计算资源极度匮乏的情况下，否则没必要在这里节省资源，另一方面，Pytorch自动求导机制中的缓存释放与重利用操作已经可以很合理的规划计算资源的使用。

Pytorch中的in-place正确性检查可以保证当我们使用in-place操作时，只要系统不报错，就说明计算的梯度是正确的。但是其具体的操作原理还需要去读一下Pytorch的源代码。Every tensor keeps a version counter, that is incremented every time it is marked dirty in any operation. When a Function saves any tensors for backward, a version counter of their containing Tensor is saved as well. Once you access self.saved_tensors it is checked, and if it is greater than the saved value an error is raised. 